# Wind Real-Time Generation from LLano Estacado (White Deer)

In [ ]:
# Use virtual environment "h2_ws" to run the program
# conda activate h2_ws

In [ ]:
# Set up libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import matplotlib.dates as mdates
from pathlib import Path
import os
import numpy as np

# Check versions
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

In [ ]:
# Define the path to the input Excel file and the sheet name
file_path = Path(
    "historical_wdwf_data/input-file-EnergyDetails_2019-10-01_2020-09-30.xlsx"
)
sheet_name = "Data_Entity"

# Read the specified range from the Excel file
wind_data = pd.read_excel(
    file_path, sheet_name=sheet_name, usecols="A:V", skiprows=1, nrows=105409
)

In [ ]:
# Ensure Date column is in datetime format (though it should already be)
wind_data.iloc[:, 1] = pd.to_datetime(wind_data.iloc[:, 1])

# Inspect and convert Time column to hh:mm:ss format if necessary
time_column = wind_data.iloc[:, 2]

# Replace '24:00' with '00:00' and add one day to the corresponding date
time_column = time_column.replace("24:00", "00:00")
wind_data.iloc[:, 1] = wind_data.iloc[:, 1] + pd.to_timedelta(
    (time_column == "00:00").astype(int), unit="D"
)

In [ ]:
# Convert the time column to hh:mm:ss format if necessary
if not time_column.str.match(r"^\d{2}:\d{2}:\d{2}$").all():
    # Assuming the time column is in a different format, e.g., hh:mm or hh:mm:ss.sss
    # Clean and convert to hh:mm:ss format
    def clean_time_string(time_str):
        # Extract the time part using regex
        match = re.search(r"\d{2}:\d{2}(:\d{2})?", time_str)
        if match:
            return match.group(0)
        return "00:00:00"  # Default value if no match found

    wind_data.iloc[:, 2] = time_column.apply(
        lambda x: pd.to_datetime(clean_time_string(x)).strftime("%H:%M:%S")
    )

# Convert Time column to timedelta and add to Date to form a complete Datetime
wind_data["Datetime"] = wind_data.iloc[:, 1] + pd.to_timedelta(wind_data.iloc[:, 2])

# Set Datetime as the index of the DataFrame
wind_data.set_index("Datetime", inplace=True)

# Sort the DataFrame by the Datetime index
wind_data.sort_index(inplace=True)

# Convert generation data in column index 6 to the opposite of its existing value
wind_data.iloc[:, 6] = wind_data.iloc[:, 6] * -1

# Rename columns to lowercase and replace spaces with underscores
wind_data.columns = wind_data.columns.str.lower().str.replace(" ", "_")

# Rename specific columns for clarity
wind_data.rename(
    columns={
        "real-time_billing_meter_quantity_[mw]": "wind_real-time_generation_mw",
        "hourly_wind_forecast_[mw]": "wind_forecast_mw",
    },
    inplace=True,
)

In [ ]:
# Convert the 5M-real-time generation in column index 10 to the opposite of its existing value
wind_data.iloc[:, 10] = wind_data.iloc[:, 10] * -1

In [ ]:
# Filter data for the specified date range
start_date = "2019-10-01"
end_date = "2019-10-08"
filtered_wind_data = wind_data.loc[start_date:end_date]

# Extract the 'real-time lmp price' column
lmp_prices = filtered_wind_data["real-time_lmp_[$/mwh]"]

# Extract the real levelized cost of hydrogen with 45V PTC (LCOH) column
# lcoh_45v = filtered_wind_data['real_levelized_cost_45v_ptc_[$/kgh2]']
# LCOH is not correct, ignoring it for now

# Calculate the average
average_lmp_price = lmp_prices.mean()
max_lmp_price = lmp_prices.max()
min_lmp_price = lmp_prices.min()

# LCOH calculations are commented out as the column is not correct
# average_lcoh_45v = lcoh_45v.mean()
# max_lcoh_45v = lcoh_45v.max()
# min_lcoh_45v = lcoh_45v.min()

In [ ]:
# Plot power generation and hourly wind forecast
plt.figure(figsize=(12, 18))  # Increase the height to accommodate all three plots

# Plot power generation (assuming it's in column index 6, which is the 7th column)
plt.subplot(2, 1, 1)
sns.lineplot(
    data=filtered_wind_data,
    x=filtered_wind_data.index,
    y=filtered_wind_data.iloc[:, 6],
    label="Power Generation",
    linestyle="-",
    color="cyan",
    linewidth=2.5,
)
sns.lineplot(
    data=filtered_wind_data,
    x=filtered_wind_data.index,
    y=filtered_wind_data["wind_forecast_mw"],
    label="Wind Forecast",
    linestyle="--",
    color="cyan",
    linewidth=2,
)
plt.title(
    "Llano Estacado Wind Farm Generation and Forecast from Oct 1, 2019 to Oct 8, 2019",
    fontsize=14,
    color="white",
)
plt.xlabel("Date and Time", fontsize=12, color="white")
plt.ylabel("Power Generation [MW]", fontsize=12, color="white")
plt.xticks(rotation=45, fontsize=10, color="white")
plt.yticks(fontsize=10, color="white")
legend = plt.legend(fontsize=12)
for text in legend.get_texts():
    text.set_color("white")

# Set the axes lines to white
ax = plt.gca()
ax.spines["bottom"].set_color("white")
ax.spines["top"].set_color("white")
ax.spines["right"].set_color("white")
ax.spines["left"].set_color("white")
ax.set_facecolor("black")

# Plot real-time LMP price (assuming it's in column index 7, which is the 8th column)
plt.subplot(2, 1, 2)
sns.lineplot(
    data=filtered_wind_data,
    x=filtered_wind_data.index,
    y=filtered_wind_data.iloc[:, 7],
    color="green",
)
plt.axhline(
    average_lmp_price,
    color="yellow",
    linestyle="--",
    linewidth=2,
    label=f"Average LMP Price: {average_lmp_price:.2f}",
)
plt.axhline(
    max_lmp_price,
    color="white",
    linestyle="--",
    linewidth=1.5,
    label=f"Max LMP Price: {max_lmp_price:.2f}",
)
plt.axhline(
    min_lmp_price,
    color="white",
    linestyle="--",
    linewidth=1.5,
    label=f"Min LMP Price: {min_lmp_price:.2f}",
)
plt.title(
    "Real-Time LMP Price from Oct 1, 2019 to Oct 8, 2019", fontsize=14, color="white"
)
plt.xlabel("Date and Time", fontsize=12, color="white")
plt.ylabel("Real-Time LMP Price [$/MWh]", fontsize=12, color="white")
plt.xticks(rotation=45, fontsize=10, color="white")
plt.yticks(fontsize=10, color="white")
legend = plt.legend(fontsize=12)
legend.get_frame().set_facecolor("none")
for text in legend.get_texts():
    text.set_color("white")

# Set the axes lines to white
ax = plt.gca()
ax.spines["bottom"].set_color("white")
ax.spines["top"].set_color("white")
ax.spines["right"].set_color("white")
ax.spines["left"].set_color("white")
ax.set_facecolor("black")

# # Plot real levelized cost 45V PTC (assuming it's in column index 19, which is the 20th column)
# plt.subplot(3, 1, 3)
# sns.lineplot(data=filtered_wind_data, x=filtered_wind_data.index, y=filtered_wind_data.iloc[:, 19], color='orange')
# plt.axhline(average_lcoh_45v, color='yellow', linestyle='--', linewidth=2, label=f'Average LCOH w/45V: {average_lcoh_45v:.2f} $/kg_H2')
# plt.axhline(max_lcoh_45v, color='white', linestyle='--', linewidth=1.5, label=f'Max LCOH w/45V: {max_lcoh_45v:.2f} $/kg_H2')
# plt.axhline(min_lcoh_45v, color='white', linestyle='--', linewidth=1.5, label=f'Min LCOH w/45V: {min_lcoh_45v:.2f} $/kg_H2')
# plt.title('Real Levelized Cost 45V PTC from Oct 1, 2019 to Oct 8, 2019', fontsize=14, color='white')
# plt.xlabel('Date and Time', fontsize=12, color='white')
# plt.ylabel('Real Levelized Cost 45V PTC [$/kg_H2]', fontsize=12, color='white')
# plt.xticks(rotation=45, fontsize=10, color='white')
# plt.yticks(fontsize=10, color='white')
# legend = plt.legend(fontsize=12)
# legend.get_frame().set_facecolor('none')
# for text in legend.get_texts():
#     text.set_color('white')

# # Set the axes lines to white
# ax = plt.gca()
# ax.spines['bottom'].set_color('white')
# ax.spines['top'].set_color('white')
# ax.spines['right'].set_color('white')
# ax.spines['left'].set_color('white')
# ax.set_facecolor('black')

# Make the legend box background clear
legend.get_frame().set_facecolor("none")
# legend.get_frame().set_edgecolor('none')

# Set the background color of the plots to black
plt.gcf().patch.set_facecolor("black")
plt.tight_layout()  # Adjust layout to prevent overlap
plt.show()

In [ ]:
# Filter data for October 2, 2019
oct_2_data = wind_data.loc["2019-10-02"]

# Plot hourly power generation and 5-min generation for October 2, 2019
plt.figure(figsize=(12, 6))

# Plot hourly power generation
sns.lineplot(data=oct_2_data, x=oct_2_data.index, y=oct_2_data["wind_real-time_generation_mw"], label="Hourly Real Generation")

# Plot 5-min generation
sns.lineplot(data=oct_2_data, x=oct_2_data.index, y=oct_2_data["5m-real-time_billing_meter_quantity_[mw]"], label="5-Min Real Generation", color="lightgreen")

# Plot forecasted generation
sns.lineplot(
    data=oct_2_data,
    x=oct_2_data.index,
    y=oct_2_data["wind_forecast_mw"],
    label="Hourly Forecasted Generation",
    color="white",
    linestyle="--",
)

plt.title(
    "Llano Estacado Wind Farm Hourly and 5-Min Generation on Oct 2, 2019",
    fontsize=14,
    color="white",
)
plt.xlabel("Time", fontsize=12, color="white")
plt.ylabel("Power Generation [MW]", fontsize=12, color="white")
plt.xticks(rotation=45, fontsize=10, color="white")
plt.yticks(fontsize=10, color="white")

# Set major ticks to hourly intervals
plt.gca().xaxis.set_major_locator(mdates.HourLocator(interval=1))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))

# Set the axes lines to white
ax = plt.gca()
ax.spines["bottom"].set_color("white")
ax.spines["top"].set_color("white")
ax.spines["right"].set_color("white")
ax.spines["left"].set_color("white")

# Set the background color of the plot to black
plt.gcf().patch.set_facecolor("black")
ax.set_facecolor("black")

# Add legend
legend = plt.legend(fontsize=12)
for text in legend.get_texts():
    text.set_color("white")

plt.tight_layout()  # Adjust layout to prevent overlap
plt.show()

# Solar PV Simulated Generation at Llano Estacado (White Deer) Wind Farm using RenewablesNinja
20 MW, 0.1 Loss, No Tracking

In [ ]:
# Load simulated solar PV data from RenewablesNinja at 20 MW, 0.1 loss, no tracking

# Load the data for 2019
solar_file_path_2019 = Path(
    "renewables_ninja_solar_data/ninja_pv_2019_20mw_0.1loss_notrack.csv"
)

solar_data_2019 = pd.read_csv(
    solar_file_path_2019, usecols=[0, 1, 2], skiprows=3, nrows=8760, header=0
)

print(solar_data_2019.columns)


# Load the data for 2020

solar_file_path_2020 = Path(
    "renewables_ninja_solar_data/ninja_pv_2020_20mw_0.1loss_notrack.csv"
)

solar_data_2020 = pd.read_csv(
    solar_file_path_2020, usecols=[0, 1, 2], skiprows=3, nrows=8784, header=0
)

print(solar_data_2020.columns)


# Concatenate the data

solar_data = pd.concat([solar_data_2019, solar_data_2020], axis=0, ignore_index=True)


print(solar_data.columns)

In [ ]:
# Convert local time column to datetime format
solar_data.iloc[:, 1] = pd.to_datetime(solar_data.iloc[:, 1])

# Set the local time column as the index using DatetimeIndex
solar_data.set_index(pd.DatetimeIndex(solar_data.iloc[:, 1]), inplace=True)

# Convert generation data from kW to MW
solar_data.iloc[:, 2] = solar_data.iloc[:, 2] / 1000

# Rename columns to match data, remove spaces, and make all lowercase
solar_data.columns = [
    "solar_date_time",
    "solar_local_time_iso_8601",
    "solar_generation_mw",
]

In [ ]:
# Filter data for the specified date range
start_date = "2019-10-01"
start_date_long_format = pd.to_datetime(start_date).strftime("%B %d, %Y")
end_date = "2019-10-08"
end_date_long_format = pd.to_datetime(end_date).strftime("%B %d, %Y")
filtered_solar_data = solar_data.loc[start_date:end_date]

In [ ]:
# Plot local time and generation
plt.figure(figsize=(12, 6))
sns.lineplot(
    data=filtered_solar_data,
    x=filtered_solar_data.index,
    y=filtered_solar_data.iloc[:, 2],
    color="orange",
    label="Solar PV Simulated Generation",
)
plt.title(
    f"Simulated Solar PV Generation from {start_date_long_format} to {end_date_long_format}",
    fontsize=14,
    color="white",
)
plt.xlabel("Date and Time", fontsize=12, color="white")
plt.ylabel("Generation [MW]", fontsize=12, color="white")
plt.xticks(rotation=45, fontsize=10, color="white")
plt.yticks(fontsize=10, color="white")

# Set major ticks to daily intervals
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=1))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%d"))

# Set the axes lines to white
ax = plt.gca()
ax.spines["bottom"].set_color("white")
ax.spines["top"].set_color("white")
ax.spines["right"].set_color("white")
ax.spines["left"].set_color("white")

# Set the background color of the plot to black
plt.gcf().patch.set_facecolor("black")
ax.set_facecolor("black")

# Customize legend font color to white
legend = plt.legend(fontsize=12)
for text in legend.get_texts():
    text.set_color("white")

plt.tight_layout()  # Adjust layout to prevent overlap
plt.show()

# Merge Data Sets

In [ ]:
# Merge the filtered wind and solar data on their datetime index using an inner join
merged_data = pd.merge(
    wind_data, solar_data, left_index=True, right_index=True, how="inner"
    # how="left"  # Uncomment this line if you want to keep all wind data even if there's no solar data
)

# Reset the index to include the datetime index as a column
merged_data.reset_index(inplace=True)

# Rename the 'index' column to 'datetime' for clarity
merged_data.rename(columns={"Datetime": "datetime_index"}, inplace=True)

# Set the 'datetime' column as the index
merged_data.set_index("datetime_index", inplace=True)

# Filter the merged data for the specified date range
filtered_merged_data = merged_data.loc[start_date:end_date]

# Copy the merged_data with only desired columns, including the datetime index
desired_columns = [
    "wind_real-time_generation_mw",
    "wind_forecast_mw",
    "solar_generation_mw",
    "real-time_lmp_[$/mwh]",
    "5m-real-time_billing_meter_quantity_[mw]",

]
filtered_merged_data = filtered_merged_data[
    desired_columns
].copy()  # Create a copy to avoid SettingWithCopyWarning

# Add a column for total generation in MW
filtered_merged_data.loc[:, "total_generation_mw"] = (
    filtered_merged_data["wind_real-time_generation_mw"]
    + filtered_merged_data["solar_generation_mw"]
)

# Plot Hourly Renewable Generation (Real Wind + Simulated Solar PV) at Llano Estacado (White Deer) Wind Farm

In [ ]:
# Set the font to Franklin Gothic and increase font size
plt.rcParams["font.family"] = "Franklin Gothic Book"
plt.rcParams["font.size"] = 14

# Set the background color to black
plt.rcParams["axes.facecolor"] = "black"
plt.rcParams["figure.facecolor"] = "black"
plt.rcParams["savefig.facecolor"] = "black"

# Plot filtered_merged_data with all columns
plt.figure(figsize=(14, 8))

# Plot real-time billing meter quantity
plt.plot(
    filtered_merged_data.index,
    filtered_merged_data["wind_real-time_generation_mw"],
    label="Real Wind Generation [MW]",
    linestyle="-",
    color="cyan",
    linewidth=2.5,
)

# Plot hourly wind forecast
plt.plot(
    filtered_merged_data.index,
    filtered_merged_data["wind_forecast_mw"],
    label="Forecasted Wind Generation [MW]",
    linestyle=":",
    color="cyan",
    linewidth=1.5,
)

# Plot solar generation
plt.plot(
    filtered_merged_data.index,
    filtered_merged_data["solar_generation_mw"],
    label="Simulated Solar Generation [MW]",
    linestyle="-",
    color="orange",
)

# Plot total generation
plt.plot(
    filtered_merged_data.index,
    filtered_merged_data["total_generation_mw"],
    label="Total Generation (Real Wind + Simulated Solar) [MW]",
    linestyle="--",
    color="white",
    linewidth=2,
)

# Adding title and labels
plt.title("Wind and Solar Generation from Oct 1, 2019 to Oct 8, 2019", color="white")
plt.xlabel("Date", color="white")
plt.ylabel("Power [MW]", color="white")

# Change the color of the tick labels and tick marks
plt.tick_params(colors="white", which="both")  # both major and minor ticks are affected

# Change the color of the axes
ax = plt.gca()  # get current axes
ax.spines["bottom"].set_color("white")
ax.spines["top"].set_color("white")
ax.spines["left"].set_color("white")
ax.spines["right"].set_color("white")

# Change the color of the legend text
legend = plt.legend()
for text in legend.get_texts():
    text.set_color("white")

# Display the plot
plt.show()

# Merge Data Sets for 1-Day

In [ ]:
from datetime import datetime

# Define the target date for filtering
target_date = "2019-10-02"

# Convert the target date to a long format string
# (e.g., "August 02, 2019")
target_date_long_format = datetime.strptime(target_date, "%Y-%m-%d").strftime("%B %d, %Y")
print(f"Target Date in Long Format: {target_date_long_format}")

# Filter wind and solar data for the target date

filtered_wind_data = wind_data.loc[target_date].drop_duplicates()

filtered_solar_data = solar_data.loc[target_date].drop_duplicates()


# Merge the filtered wind and solar data on their datetime index using an inner join

merged_data = pd.merge(
    filtered_wind_data,
    filtered_solar_data,
    left_index=True,
    right_index=True,
    how="inner",
)


# Reset the index to include the datetime index as a column

merged_data.reset_index(inplace=True)


# Rename the 'Datetime' column to 'datetime_index' for clarity

merged_data.rename(columns={"Datetime": "datetime_index"}, inplace=True)

# Set the 'datetime_index' column as the index
merged_data.set_index("datetime_index", inplace=True)


# Copy the merged_data with only desired columns, including the datetime index

desired_columns = [
    "wind_real-time_generation_mw",
    "wind_forecast_mw",
    "solar_generation_mw",
    "real-time_lmp_[$/mwh]",
]

filtered_merged_data = merged_data[
    desired_columns
].copy()  # Create a copy to avoid SettingWithCopyWarning


# Add a column for total generation in MW
filtered_merged_data.loc[:, "total_generation_mw"] = (
    filtered_merged_data["wind_real-time_generation_mw"]
    + filtered_merged_data["solar_generation_mw"]
)

In [ ]:
# Set the font to Franklin Gothic and increase font size
plt.rcParams["font.family"] = "Franklin Gothic Book"
plt.rcParams["font.size"] = 14

# Set the background color to black
plt.rcParams["axes.facecolor"] = "black"
plt.rcParams["figure.facecolor"] = "black"
plt.rcParams["savefig.facecolor"] = "black"

# Make the figure wider
plt.figure(figsize=(16, 6))  # Adjust the width to 16 for a wider plot

# Plot real-time billing meter quantity
plt.plot(
    filtered_merged_data.index,
    filtered_merged_data["wind_real-time_generation_mw"],
    label="Real Wind Generation [MW]",
    linestyle="-",
    color="cyan",
    linewidth=2.5,
)

# Plot hourly wind forecast
plt.plot(
    filtered_merged_data.index,
    filtered_merged_data["wind_forecast_mw"],
    label="Forecasted Wind Generation [MW]",
    linestyle=":",
    color="cyan",
    linewidth=1.5,
)

# Plot solar generation
plt.plot(
    filtered_merged_data.index,
    filtered_merged_data["solar_generation_mw"],
    label="Simulated Solar Generation [MW]",
    linestyle="-",
    color="orange",
)

# Plot total generation
plt.plot(
    filtered_merged_data.index,
    filtered_merged_data["total_generation_mw"],
    label="Total Generation (Real Wind + Simulated Solar) [MW]",
    linestyle="--",
    color="white",
    linewidth=2,
)

# Adding title and labels
plt.title(f"Wind and Solar Generation for {target_date_long_format}", color="white")
plt.xlabel("Date", color="white")
plt.ylabel("Power [MW]", color="white")

# Change the color of the tick labels and tick marks
plt.tick_params(colors="white", which="both")  # both major and minor ticks are affected

# Change the color of the axes
ax = plt.gca()  # get current axes
ax.spines["bottom"].set_color("white")
ax.spines["top"].set_color("white")
ax.spines["left"].set_color("white")
ax.spines["right"].set_color("white")

# Set the date format on the x-axis to show hours in 00:00 format
ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
ax.xaxis.set_major_locator(
    mdates.HourLocator(interval=1)
)  # Set tick marks for every hour

# Rotate x-axis labels by 45 degrees
plt.xticks(rotation=45)

# Change the color of the legend text and make it smaller
legend = plt.legend(fontsize=12)  # Set legend font size to 12
for text in legend.get_texts():
    text.set_color("white")

# Display the plot
plt.show()

# Generate Electrolyzer Simulation Data

In [ ]:
# Define electrolyzer load constraints (based on H35 Simple Fuel Unit Data)
steady_state_load = 60  # kW
run_idle_load = 0.65  # kW
warm_up_time = 60 / 60  # hour (11:13 to 12:13 on December 16, 2024)
fast_start_up_time = 10 / 60  # hour (13:50 to 14:00 on January 29, 2025)
ramp_down_time = 12 / 60  # hour (15:02 to 15:14 on December 16, 2024)
warm_up_rate = (
    steady_state_load - run_idle_load
) / warm_up_time  # kW/h, required on 1st start-up and if run idle time is > 4 hours
fast_start_up_rate = (
    steady_state_load - run_idle_load
) / fast_start_up_time  # kW/h, required if run idle time is < 4 hours
ramp_down_rate = (steady_state_load - run_idle_load) / ramp_down_time  # kW/h
hydrogen_production_rate = (
    1 / 63.8
)  # kg/kWh, from H35 Simple Fuel Unit Data on steady state operation on December 16, 2024


## Define Electrolyzer Operational Constraints

- The electrolyzer load starts at `run_idle_load`.
- If generation is greater than `steady_state_load`, the electrolyzer will ramp up using `warm_up_rate`.
- If generation is less than `steady_state_load`, the electrolyzer will either ramp up or down based on the conditions of the current load.
- The electrolyzer cannot operate at more than `steady_state_load`; any excess generation is sold to the grid.
- The electrolyzer cannot operate at less than `run_idle_load`; any deficit generation is purchased from the grid.
- Hydrogen is only produced when the electrolyzer is running at `steady_state_load`.
- Hydrogen is not produced when the electrolyzer is operating at less than `steady_state_load`.

In [ ]:
# Extract the columns for total generation, solar generation, wind generation, and wind forecast
total_generation_mw = filtered_merged_data["total_generation_mw"].values
solar_generation_mw = filtered_merged_data["solar_generation_mw"].values
wind_generation_mw = filtered_merged_data["wind_real-time_generation_mw"].values
wind_forecast_mw = filtered_merged_data["wind_forecast_mw"].values

# Convert MW to kW and apply scaling factor
conversion_factor = 1  # Conversion factor from MW to kW
scaling_factor = 8  # % generation scaling factor to match electrolyzer load

# Calculate the scaled total generation in kW
scaled_total_generation_kw = (
    total_generation_mw * conversion_factor * (scaling_factor/100.0)
)  # Convert MW to kW and apply scaling factor

# Calculate the scaled solar generation in kW
scaled_solar_generation_kw = (
    solar_generation_mw * conversion_factor * (scaling_factor/100.0)
)

# Calculate the scaled wind generation in kW
scaled_wind_generation_kw = (
    wind_generation_mw * conversion_factor * (scaling_factor/100.0)
)

# Calculate the scaled wind forecast in kW
scaled_wind_forecast_kw = (
    wind_forecast_mw * conversion_factor * (scaling_factor/100.0)
)

In [ ]:
# Plot scaled_solar_generation_kw and scaled_wind_generation_kw
plt.figure(figsize=(12, 6))

# Plot scaled solar generation
plt.plot(
    filtered_merged_data.index,
    scaled_solar_generation_kw,
    label="Scaled Solar Generation [kW]",
    linestyle="-",
    color="orange",
    linewidth=2.5,
)

# Plot scaled wind generation
plt.plot(
    filtered_merged_data.index,
    scaled_wind_generation_kw,
    label="Scaled Wind Generation [kW]",
    linestyle="-",
    color="cyan",
    linewidth=2.5,
)

# Plot scaled wind forecast
plt.plot(
    filtered_merged_data.index,
    scaled_wind_forecast_kw,
    label="Scaled Wind Forecast [kW]",
    linestyle="--",
    color="cyan",
    linewidth=1.5,
)

# Plot scaled total generation
plt.plot(
    filtered_merged_data.index,
    scaled_total_generation_kw,
    label="Scaled Total Generation [kW]",
    linestyle="-",
    color="white",
    linewidth=2.5,
)

# Add axis labels
plt.xlabel("Datetime", fontsize=12, color="white")
plt.ylabel("Scaled Generation [kW]", fontsize=12, color="white")

# Add title
plt.title("Scaled Solar and Wind Generation Over Time", fontsize=14, color="white")

# Customize tick marks
plt.xticks(rotation=45, fontsize=10, color="white")
plt.yticks(fontsize=10, color="white")

# Set the axes lines to white
ax = plt.gca()
ax.spines["bottom"].set_color("white")
ax.spines["top"].set_color("white")
ax.spines["right"].set_color("white")
ax.spines["left"].set_color("white")

# Set the background color of the plot to black
plt.gcf().patch.set_facecolor("black")
ax.set_facecolor("black")

# Add legend
legend = plt.legend(fontsize=12, loc="upper left")
for text in legend.get_texts():
    text.set_color("white")

plt.tight_layout()  # Adjust layout to prevent overlap
plt.show()

## Simple Fuel Unit (SFU) Electrolyzer Load Simulation
Simulating 2 electrolyzer in operation

In [ ]:
# Function to simulate electrolyzer load and hydrogen generation with constraints for two electrolyzers
def simulate_two_electrolyzers_operation(
    total_generation_kw,
    steady_state_load,
    run_idle_load,
    warm_up_rate,
    fast_start_up_rate,
    ramp_down_rate,
    hydrogen_production_rate,
):
    """
    Simulates the operation of two electrolyzers based on generation constraints and calculates hydrogen production.

    Args:
        total_generation_kw (array): Array of total generation values in kW.
        steady_state_load (float): Steady-state load of one electrolyzer in kW.
        run_idle_load (float): Run idle load of one electrolyzer in kW.
        warm_up_rate (float): Warm-up rate in kW/h.
        fast_start_up_rate (float): Fast start-up rate in kW/h.
        ramp_down_rate (float): Ramp-down rate in kW/h.
        hydrogen_production_rate (float): Hydrogen production rate in kg/kWh.

    Returns:
        dict: A dictionary containing simulated load, hydrogen production, and excess/deficit energy.
    """
    simulated_load_e1 = []
    simulated_load_e2 = []
    hydrogen_production = []
    excess_energy = []
    deficit_energy = []

    current_load_e1 = run_idle_load  # Initialize electrolyzer 1 load at run_idle_load
    current_load_e2 = run_idle_load  # Initialize electrolyzer 2 load at run_idle_load

    idle_time_counter_e1 = 0  # Counter for idle time of electrolyzer 1
    idle_time_counter_e2 = 0  # Counter for idle time of electrolyzer 2
    time_step = 1  # Assuming each time step is 1 hour

    for generation in total_generation_kw:
        # Check if electrolyzer 1 and 2 are idle and update their idle time counters
        # Electrolyzer 1 (E1)
        if current_load_e1 <= run_idle_load + 1e-6:
            idle_time_counter_e1 += time_step
        else:
            idle_time_counter_e1 = 0

        # Electrolyzer 2 (E2)
        if current_load_e2 <= run_idle_load + 1e-6:
            idle_time_counter_e2 += time_step
        else:
            idle_time_counter_e2 = 0

        if generation >= 2 * steady_state_load:
            # Both electrolyzers operate at steady-state load
            # E1
            if current_load_e1 < steady_state_load:
                ramp_rate_e1 = (
                    fast_start_up_rate
                    if idle_time_counter_e1 < 4
                    else warm_up_rate
                )
                ramp_needed_e1 = steady_state_load - current_load_e1
                ramp_time_e1 = ramp_needed_e1 / ramp_rate_e1 # hours to reach steady state
                if ramp_time_e1 < 1:
                    # Part of hour ramping, part at steady state
                    hydrogen_e1 = steady_state_load * (1 - ramp_time_e1) * hydrogen_production_rate
                    current_load_e1 = steady_state_load
                else:
                    # Full hour ramping
                    hydrogen_e1 = 0
                    current_load_e1 = min(current_load_e1 + ramp_rate_e1, steady_state_load)
                idle_time_counter_e1 = 0  # Reset idle time counter
            else:
                # Already at steady state
                hydrogen_e1 = steady_state_load * 1 * hydrogen_production_rate # kg_H2: kW * 1 hour * kg/kWh
                current_load_e1 = steady_state_load
                idle_time_counter_e1 = 0  # Reset idle time counter

            # E2
            if current_load_e2 < steady_state_load:
                ramp_rate_e2 = (
                    fast_start_up_rate
                    if idle_time_counter_e2 < 4
                    else warm_up_rate
                )
                ramp_needed_e2 = steady_state_load - current_load_e2
                ramp_time_e2 = ramp_needed_e2 / ramp_rate_e2 # hours to reach steady state
                if ramp_time_e2 < 1:
                    # Part of hour ramping, part at steady state
                    hydrogen_e2 = steady_state_load * (1 - ramp_time_e2) * hydrogen_production_rate
                    current_load_e2 = steady_state_load
                else:
                    # Full hour ramping
                    hydrogen_e2 = 0
                    current_load_e2 = min(current_load_e2 + ramp_rate_e2, steady_state_load)
                idle_time_counter_e2 = 0 # Reset idle time counter
            else:
                # Already at steady state
                hydrogen_e2 = steady_state_load * 1 * hydrogen_production_rate # kg_H2: kW * 1 hour * kg/kWh
                current_load_e2 = steady_state_load
                idle_time_counter_e2 = 0

            # Total hydrogen production
            hydrogen_production.append(hydrogen_e1 + hydrogen_e2)

            # Excess energy is sold to the grid
            excess_energy.append(generation - (current_load_e1 + current_load_e2))
            deficit_energy.append(0)

        elif steady_state_load <= generation < 2 * steady_state_load:
            # Only E1 operates at steady-state load
            if current_load_e1 < steady_state_load:
                ramp_rate_e1 = (
                    fast_start_up_rate
                    if idle_time_counter_e1 < 4
                    else warm_up_rate
                )
                ramp_needed_e1 = steady_state_load - current_load_e1
                ramp_time_e1 = ramp_needed_e1 / ramp_rate_e1 # hours to reach steady state
                if ramp_time_e1 < 1:
                    # Part of hour ramping, part at steady state
                    hydrogen_production_e1 = steady_state_load * (1 - ramp_time_e1) * hydrogen_production_rate
                    current_load_e1 = steady_state_load
                else:
                    # Full hour ramping
                    hydrogen_production_e1 = 0
                    current_load_e1 = min(current_load_e1 + ramp_rate_e1, steady_state_load)
                idle_time_counter_e1 = 0
            else:
                # Already at steady state
                hydrogen_production_e1 = steady_state_load * 1 * hydrogen_production_rate # kg_H2: kW * 1 hour * kg/kWh
                current_load_e1 = steady_state_load
                idle_time_counter_e1 = 0

            # E2 remains idle
            if current_load_e2 > run_idle_load:
                current_load_e2 = max(current_load_e2 - ramp_down_rate, run_idle_load)
            else:
                current_load_e2 = run_idle_load

            hydrogen_production.append(hydrogen_e1)

            # Excess energy is sold to the grid
            excess_energy.append(generation - current_load_e1)
            deficit_energy.append(0)

        elif run_idle_load <= generation < steady_state_load:
            # Generation is insufficient for steady-state load
            # E1 ramps down to run idle load
            if current_load_e1 > generation:
                ramp_needed_e1 = current_load_e1 - run_idle_load
                ramp_time_e1 = ramp_needed_e1 / ramp_down_rate if ramp_needed_e1 > 0 else 1
                if ramp_time_e1 < 1:
                    # Part of hour ramping, part at run idle load
                    current_load_e1 = run_idle_load
                    hydrogen_e1 = 0
                else:
                    # Full hour ramping
                    hydrogen_e1 = 0
                    current_load_e1 = max(current_load_e1 - ramp_down_rate, run_idle_load)
                current_load_e1 = max(current_load_e1 - ramp_down_rate, run_idle_load)
            else:
                current_load_e1 = run_idle_load
                hydrogen_e1 = 0

            # E2 ramps down to run idle load
            if current_load_e2 > run_idle_load:
                ramp_needed_e2 = current_load_e2 - run_idle_load
                ramp_time_e2 = ramp_needed_e2 / ramp_down_rate if ramp_needed_e2 > 0 else 1
                if ramp_time_e2 < 1:
                    # Part of hour ramping, part at run idle load
                    current_load_e2 = run_idle_load
                    hydrogen_e2 = 0
                else:
                    # Full hour ramping
                    hydrogen_e2 = 0
                    current_load_e2 = max(current_load_e2 - ramp_down_rate, run_idle_load)
                current_load_e2 = max(current_load_e2 - ramp_down_rate, run_idle_load)
            else:
                current_load_e2 = run_idle_load
                hydrogen_e2 = 0

            hydrogen_production.append(hydrogen_e1 + hydrogen_e2)

            # No excess or deficit energy
            excess_energy.append(0)
            deficit_energy.append(0)

        else:
            # Generation is below run idle load
            if current_load_e1 > run_idle_load:
                current_load_e1 = max(current_load_e1 - ramp_down_rate, run_idle_load)
            else:
                current_load_e1 = run_idle_load

            if current_load_e2 > run_idle_load:
                current_load_e2 = max(current_load_e2 - ramp_down_rate, run_idle_load)
            else:
                current_load_e2 = run_idle_load

            hydrogen_production.append(0)

            # Deficit energy is purchased from the grid
            excess_energy.append(0)
            deficit_energy.append(run_idle_load - generation)

        # Append the current loads to the simulated load lists
        simulated_load_e1.append(current_load_e1)
        simulated_load_e2.append(current_load_e2)

    return {
        "simulated_load_e1": simulated_load_e1,
        "simulated_load_e2": simulated_load_e2,
        "hydrogen_production": hydrogen_production,
        "excess_energy": excess_energy,
        "deficit_energy": deficit_energy,
    }


In [ ]:
# Call the function to simulate two electrolyzers operation
results_two_electrolyzers = simulate_two_electrolyzers_operation(
    total_generation_kw=scaled_total_generation_kw,
    steady_state_load=steady_state_load,
    run_idle_load=run_idle_load,
    warm_up_rate=warm_up_rate,
    fast_start_up_rate=fast_start_up_rate,
    ramp_down_rate=ramp_down_rate,
    hydrogen_production_rate=hydrogen_production_rate,
)

# Access results
simulated_load_e1 = results_two_electrolyzers["simulated_load_e1"]
simulated_load_e2 = results_two_electrolyzers["simulated_load_e2"]
hydrogen_production = results_two_electrolyzers["hydrogen_production"]
excess_energy = results_two_electrolyzers["excess_energy"]
deficit_energy = results_two_electrolyzers["deficit_energy"]
total_simulated_load = np.array(simulated_load_e1) + np.array(simulated_load_e2)

In [ ]:
# Plot the simulated load, generation, and hydrogen production
# Plot the original and simulated generation data
fig, ax1 = plt.subplots(figsize=(12, 6))

# Extract the 'real_time_lmp_price' column
real_time_lmp_price = filtered_merged_data["real-time_lmp_[$/mwh]"]

# # Extract the '5m-real-time_billing_meter_quantity_[mw]' column
# real_time_billing_meter_quantity = filtered_merged_data[
#     "5m-real-time_billing_meter_quantity_[mw]"
# ]

# Plot total generation and simulated load on the primary y-axis
sns.lineplot(
    x=filtered_merged_data.index,
    y=scaled_total_generation_kw,
    label="Scaled Total Generation (Real Wind + Simulated Solar) (KW)",
    color="blue",
    # linestyle="",
    ax=ax1,
)
sns.lineplot(
    x=filtered_merged_data.index,
    y=simulated_load_e1,
    label="Simulated Electrolyzer 1 Load (KW)",
    color="red",
    ax=ax1,
)

sns.lineplot(
    x=filtered_merged_data.index,
    y=simulated_load_e2,
    label="Simulated Electrolyzer 2 Load (KW)",
    color="orange",
    ax=ax1,
)

sns.lineplot(
    x=filtered_merged_data.index,
    y=total_simulated_load,
    label="Total Simulated Electrolyzer Load (KW)",
    color="white",
    linestyle="--",
    ax=ax1,
)

# sns.lineplot(
#     x=filtered_merged_data.index,
#     y=scaled_wind_forecast_kw,
#     label="Scaled Forecasted Wind Generation (KW)",
#     linestyle=":",
#     color="white",
#     ax=ax1,
# )

# sns.lineplot(
#     x=filtered_merged_data.index,
#     y=real_time_billing_meter_quantity,
#     label="5-Min Real Wind Generation (KW)",
#     linestyle=":",
#     color="cyan",
#     ax=ax1,
# )

# Shade the area between the curves
ax1.fill_between(
    filtered_merged_data.index,
    scaled_total_generation_kw,
    total_simulated_load,
    where=(scaled_total_generation_kw > total_simulated_load),
    interpolate=True,
    color="green",
    alpha=0.4,
    label="Energy Excess Sold to Grid (KWh)",
)
ax1.fill_between(
    filtered_merged_data.index,
    scaled_total_generation_kw,
    total_simulated_load,
    where=(total_simulated_load > scaled_total_generation_kw),
    interpolate=True,
    color="red",
    alpha=0.4,
    label="Energy Deficit Purchased from Grid (KWh)",
)

# Set labels and title for the primary y-axis
ax1.set_title(
    f"Total Generation vs Simulated Load for {target_date_long_format}",
    fontsize=14,
    color="white",
)
ax1.set_xlabel("Time", fontsize=12, color="white")
ax1.set_ylabel("Power (KW)", fontsize=12, color="white")
ax1.tick_params(axis="x", rotation=45, colors="white")
ax1.tick_params(axis="y", colors="white")

# Set the axes lines to white
ax1.spines["bottom"].set_color("white")
ax1.spines["top"].set_color("white")
ax1.spines["right"].set_color("white")
ax1.spines["left"].set_color("white")

# Set the date format on the x-axis to show hours in 00:00 format
ax1.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
ax1.xaxis.set_major_locator(
    mdates.HourLocator(interval=1)
)  # Set tick marks for every hour

# # Create a secondary y-axis to plot the real-time LMP price
# ax2 = ax1.twinx()
# sns.lineplot(
#     x=filtered_merged_data.index,
#     y=real_time_lmp_price,
#     label="Real-Time LMP Price ($/MWh)",
#     color="yellow",
#     ax=ax2,
# )
# ax2.set_ylabel("Real-Time LMP Price ($/MWh)", fontsize=12, color="yellow")
# ax2.tick_params(axis="y", colors="yellow")

# Combine legends from both y-axes and place them below the plot
lines, labels = ax1.get_legend_handles_labels()
# lines2, labels2 = ax2.get_legend_handles_labels()
legend = ax1.legend(
    # lines + lines2,
    # labels + labels2,
    loc="upper center",
    bbox_to_anchor=(0.5, -0.16),
    fontsize=12,
    ncol=2,
)

# # Remove the legend box on the top right corner
# ax2.get_legend().remove()

# Set the legend text color to white
for text in legend.get_texts():
    text.set_color("white")

# Show the plot
plt.show()

In [ ]:
# Plot the simulated load and hydrogen production
fig, ax1 = plt.subplots(figsize=(12, 6))

# Plot simulated load on the left y-axis
ax1.plot(
    filtered_merged_data.index,
    total_simulated_load,
    label="Total Simulated Load (KW)",
    linestyle="-",
    color="cyan",
    linewidth=2.5,
)
ax1.set_xlabel("Time", fontsize=14, color="white")
ax1.set_ylabel("Simulated Load [KW]", fontsize=14, color="white")
ax1.tick_params(axis="x", rotation=45, colors="white")
ax1.tick_params(axis="y", colors="white")
ax1.spines["bottom"].set_color("white")
ax1.spines["top"].set_color("white")
ax1.spines["right"].set_color("white")
ax1.spines["left"].set_color("white")

# Set the x-axis to display hours in 00:00 format
ax1.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
ax1.xaxis.set_major_locator(mdates.HourLocator(interval=1))

# Set the background color of the plot to black
plt.gcf().patch.set_facecolor("black")
ax1.set_facecolor("black")

# Create a second y-axis for hydrogen production
ax2 = ax1.twinx()
ax2.plot(
    filtered_merged_data.index,
    hydrogen_production,
    label="Hydrogen Production (kg)",
    linestyle="--",
    color="yellow",
    linewidth=2,
)
ax2.set_ylabel("Hydrogen Production [kg]", fontsize=12, color="yellow")
ax2.tick_params(axis="y", colors="yellow")

# Combine legends from both axes
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
legend = ax1.legend(
    lines1 + lines2,
    labels1 + labels2,
    loc="upper center",
    bbox_to_anchor=(0.5, -0.15),
    fontsize=14,
    ncol=2,
)

# Set the legend text color to white
for text in legend.get_texts():
    text.set_color("white")

plt.title(f"Simulated Load and Hydrogen Production for {target_date_long_format}", fontsize=14, color="white")
plt.tight_layout()  # Adjust layout to prevent overlap
plt.show()

# Print total hydrogen production
total_hydrogen_production = sum(hydrogen_production)  # kg
print(f"Total Hydrogen Production: {total_hydrogen_production:.2f} kg")

# Calculate Simulated Energy Deficits & Excess and Associated Cost

Note: Revision required as of May 6, 2025

In [ ]:
from scipy.integrate import simpson

# Initialize a list to store the area differences for each hour
negative_hourly_area_differences = []

# Loop through each hour and calculate the area difference if the condition is met
for i in range(1, len(scaled_total_generation_kw)):
    if simulated_load_kw[i] > scaled_total_generation_kw[i]:
        # Calculate the area under the curve for the original total generation data for the current hour
        area_original_hour = simpson(total_generation_mw[i - 1 : i + 1], dx=1)

        # Calculate the area under the curve for the simulated total generation data for the current hour
        area_simulated_hour = simpson(simulated_load[i - 1 : i + 1], dx=1)

        # Calculate the difference in the areas for the current hour
        area_difference_hour = area_simulated_hour - area_original_hour

        # Append the area difference to the list
        negative_hourly_area_differences.append((i, area_difference_hour))

# Print the hourly area differences
for hour, diff in negative_hourly_area_differences:
    print(f"At hour {hour}:00, the area difference: {diff:.2f} MWh")

In [ ]:
# Calculate the total hourly area differences that would require energy purchase from the grid
negative_total_area_difference = sum(
    diff for _, diff in negative_hourly_area_differences if diff > 0
)
print(
    f"This wind and solar power simulation would require {negative_total_area_difference:.2f} MWh of energy to be purchased from the grid."
)

# Calculate the cost of this energy purchase based on the corresponding hourly real-time LMP price
total_cost = sum(
    diff * oct_2_merged_data.iloc[hour]["real-time_lmp_[$/mwh]"]
    for hour, diff in negative_hourly_area_differences
    if diff > 0
)
print(f"The total cost of the energy purchase from the grid is ${total_cost:.2f}")

In [ ]:
from scipy.integrate import simpson

# Initialize a list to store the area differences for each hour
positive_hourly_area_differences = []

# Loop through each hour and calculate the area difference if the condition is met
for i in range(1, len(scaled_total_generation_kw)):
    if scaled_total_generation_kw[i] > simulated_load_kw[i]:
        # Calculate the area under the curve for the original total generation data for the current hour
        area_original_hour = simpson(scaled_total_generation_kw[i - 1 : i + 1], dx=1)

        # Calculate the area under the curve for the simulated total generation data for the current hour
        area_simulated_hour = simpson(simulated_load_kw[i - 1 : i + 1], dx=1)

        # Calculate the difference in the areas for the current hour
        area_difference_hour = area_original_hour - area_simulated_hour

        # Append the area difference to the list
        positive_hourly_area_differences.append((i, area_difference_hour))

# Print the hourly area differences
for hour, diff in positive_hourly_area_differences:
    print(f"At hour {hour}:00, the area difference: {diff:.2f} MWh")

In [ ]:
# Calculate the total hourly area differences that would require energy sale to the grid
positive_total_area_difference = sum(
    diff for _, diff in positive_hourly_area_differences if diff > 0
)
print(
    f"This wind and solar power simulation would require {positive_total_area_difference:.2f} MWh of energy to be sold to the grid."
)

# Calculate the revenue from this energy sale based on the corresponding hourly real-time LMP price
total_revenue = sum(
    diff * filtered_merged_data.iloc[hour]["real-time_lmp_[$/mwh]"]
    for hour, diff in positive_hourly_area_differences
    if diff > 0
)
print(f"The total revenue from energy sales to the grid is ${total_revenue:.2f}")

In [ ]:
# Calculate the net energy from the energy sales and purchases
net_energy = positive_total_area_difference - negative_total_area_difference
print(
    f"The net energy from energy sales and purchases is {net_energy:.2f} MWh."
)  # Calculate the net revenue from the energy sales and purchases
if net_energy > 0:
    print(f"Sold more energy than purchased from the grid.")
else:
    print(f"Purchased more energy than sold to the grid.")

net_revenue = total_revenue - total_cost
print(f"The net revenue from energy sales and purchases is ${net_revenue:.2f}.")
if net_revenue > 0:
    print(f"Profited from the energy sales.")

Determine during which times of the day is LMP price highest using data set

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming merged_data is already loaded as a DataFrame
# merged_data = pd.read_csv('path_to_your_dataset.csv')

# Extract the hour from the datetime index
merged_data["hour"] = merged_data.index.hour

# Calculate the correlation between the hour and the LMP price
correlation = merged_data[["hour", "real-time_lmp_[$/mwh]"]].corr()
print("Correlation between hour and LMP price:")
print(correlation)

# Prepare the data for machine learning
X = merged_data[["hour"]]
y = merged_data["real-time_lmp_[$/mwh]"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Train a linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# # Plot the actual vs predicted LMP prices
# plt.figure(figsize=(10, 6))
# sns.scatterplot(x=X_test['hour'], y=y_test, label='Actual', color='blue')
# sns.scatterplot(x=X_test['hour'], y=y_pred, label='Predicted', color='red')
# plt.title('Actual vs Predicted LMP Prices', color='white')
# plt.xlabel('Hour of the Day', color='white')
# plt.ylabel('Real-Time LMP Price [$/MWh]', color='white')

# # Customize plot appearance
# ax = plt.gca()
# ax.spines['bottom'].set_color('white')
# ax.spines['top'].set_color('white')
# ax.spines['right'].set_color('white')
# ax.spines['left'].set_color('white')

# # Set the legend text color to white and background to transparent
# legend = plt.legend()
# for text in legend.get_texts():
#     text.set_color('white')
# legend.get_frame().set_facecolor('none')

# plt.show()

# Identify outliers in the actual data
outliers = y_test[(y_test - y_pred).abs() > 2 * mse**0.5]

# Plot the outliers with labels
plt.figure(figsize=(10, 6))
sns.scatterplot(x=X_test["hour"], y=y_test, label="Actual", color="blue")
sns.scatterplot(x=X_test["hour"], y=y_pred, label="Predicted", color="red")
for idx, value in outliers.items():
    plt.text(
        X_test.loc[idx, "hour"],
        value,
        idx.strftime("%b-%d"),
        color="yellow",
        fontsize=10,
        ha="right",
    )

# Extract the first and last date from the merged_data index
from_date = merged_data.index.min().strftime("%Y-%m-%d")
to_date = merged_data.index.max().strftime("%Y-%m-%d")

plt.title(
    f"Actual vs Predicted Real-Time LMP Prices from {from_date} to {to_date}",
    color="white",
)
plt.xlabel("Hour of the Day", color="white")
plt.ylabel("Real-Time LMP Price [$/MWh]", color="white")

# Customize plot appearance
ax = plt.gca()
ax.spines["bottom"].set_color("white")
ax.spines["top"].set_color("white")
ax.spines["right"].set_color("white")
ax.spines["left"].set_color("white")

# Set the legend text color to white and background to transparent
legend = plt.legend()
for text in legend.get_texts():
    text.set_color("white")
legend.get_frame().set_facecolor("none")

plt.show()

In [ ]:
# Determine the times of the day when the LMP price is typically highest
average_lmp_by_hour = merged_data.groupby("hour")["real-time_lmp_[$/mwh]"].mean()

# Plot the average LMP price by hour
plt.figure(figsize=(10, 6))
sns.lineplot(x=average_lmp_by_hour.index, y=average_lmp_by_hour.values, color="blue")
plt.title("Average Real-Time LMP Price by Hour of the Day", color="white")
plt.xlabel("Hour of the Day", color="white")
plt.ylabel("Average Real-Time LMP Price [$/MWh]", color="white")
plt.xticks(color="white")
plt.yticks(color="white")

# Add labels to points on the line for every hour
for hour, price in average_lmp_by_hour.items():
    plt.text(
        hour,
        price,
        f"${price:.0f}",
        color="white",
        fontsize=11,
        ha="center",
        va="bottom",
    )

# Set the axes lines to white
ax = plt.gca()
ax.spines["bottom"].set_color("white")
ax.spines["top"].set_color("white")
ax.spines["right"].set_color("white")
ax.spines["left"].set_color("white")

# Set the background color of the plot to black
plt.gcf().patch.set_facecolor("black")
ax.set_facecolor("black")

plt.show()